<a href="https://colab.research.google.com/github/Krazahra/Machine-Learning/blob/main/Tugas_12_GoogleNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Identitas
Nama: Kinanti Rahayu Az-Zahra

Kelas: TK-45-04

NIM: 1103210052

Dataset: https://www.tensorflow.org/datasets/catalog/mnist

Dokumentasi GPT: https://chatgpt.com/share/4980e1f0-6366-497c-a020-a85d9f7b411d

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [2]:
# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [3]:
# Preprocess the data
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

In [4]:
# Convert labels to categorical one-hot encoding
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

In [5]:
# Define GoogleNet (Inception) model
def create_googlenet_model(input_shape):
    input = layers.Input(shape=input_shape)

    conv1 = layers.Conv2D(64, (7, 7), padding='same', activation='relu')(input)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(conv1)

    inception2a = inception_module(pool1, 64, 96, 128, 16, 32, 32)
    inception2b = inception_module(inception2a, 128, 128, 192, 32, 96, 64)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(inception2b)

    inception3a = inception_module(pool2, 192, 96, 208, 16, 48, 64)
    inception3b = inception_module(inception3a, 160, 112, 224, 24, 64, 64)

    flatten = layers.Flatten()(inception3b)
    dense1 = layers.Dense(128, activation='relu')(flatten)
    output = layers.Dense(10, activation='softmax')(dense1)

    model = models.Model(inputs=input, outputs=output)
    return model

def inception_module(prev_layer, conv1_out, conv3_reduce_out, conv3_out, conv5_reduce_out, conv5_out, pool_proj_out):
    conv1 = layers.Conv2D(conv1_out, (1, 1), padding='same', activation='relu')(prev_layer)

    conv3_reduce = layers.Conv2D(conv3_reduce_out, (1, 1), padding='same', activation='relu')(prev_layer)
    conv3 = layers.Conv2D(conv3_out, (3, 3), padding='same', activation='relu')(conv3_reduce)

    conv5_reduce = layers.Conv2D(conv5_reduce_out, (1, 1), padding='same', activation='relu')(prev_layer)
    conv5 = layers.Conv2D(conv5_out, (5, 5), padding='same', activation='relu')(conv5_reduce)

    pool_proj = layers.MaxPooling2D((3, 3), strides=(1, 1), padding='same')(prev_layer)
    pool_proj = layers.Conv2D(pool_proj_out, (1, 1), padding='same', activation='relu')(pool_proj)

    output = layers.concatenate([conv1, conv3, conv5, pool_proj], axis=-1)
    return output


In [6]:
# Create GoogleNet model
input_shape = (28, 28, 1)
model = create_googlenet_model(input_shape)
model.summary()

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 28, 28, 1)]          0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 28, 28, 64)           3200      ['input_1[0][0]']             
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 14, 14, 64)           0         ['conv2d[0][0]']              
 D)                                                                                               
                                                                                                  
 conv2d_2 (Conv2D)           (None, 14, 14, 96)           6240      ['max_pooling2d[0][0]']   

In [7]:
# Define early stopping callback
early_stop = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

# Train the model with early stopping
history = model.fit(x_train.reshape(-1, 28, 28, 1), y_train, epochs=20, batch_size=128,
                    validation_data=(x_test.reshape(-1, 28, 28, 1), y_test),
                    callbacks=[early_stop])

Epoch 1/20
469/469 [==============================] - 42s 57ms/step - loss: 0.1598 - accuracy: 0.9487 - val_loss: 0.0346 - val_accuracy: 0.9878
Epoch 2/20
469/469 [==============================] - 23s 49ms/step - loss: 0.0415 - accuracy: 0.9873 - val_loss: 0.0344 - val_accuracy: 0.9890
Epoch 3/20
469/469 [==============================] - 24s 50ms/step - loss: 0.0297 - accuracy: 0.9905 - val_loss: 0.0253 - val_accuracy: 0.9914
Epoch 4/20
469/469 [==============================] - 23s 49ms/step - loss: 0.0225 - accuracy: 0.9934 - val_loss: 0.0279 - val_accuracy: 0.9914
Epoch 5/20
469/469 [==============================] - 23s 49ms/step - loss: 0.0220 - accuracy: 0.9930 - val_loss: 0.0308 - val_accuracy: 0.9902
Epoch 6/20
469/469 [==============================] - 23s 49ms/step - loss: 0.0170 - accuracy: 0.9944 - val_loss: 0.0265 - val_accuracy: 0.9915
Epoch 7/20
469/469 [==============================] - 23s 49ms/step - loss: 0.0164 - accuracy: 0.9946 - val_loss: 0.0272 - val_accuracy:

In [8]:
# Evaluate the model
test_loss, test_acc = model.evaluate(x_test.reshape(-1, 28, 28, 1), y_test)
print(f'Test accuracy: {test_acc:.4f}')

313/313 [==============================] - 3s 7ms/step - loss: 0.0323 - accuracy: 0.9934
Test accuracy: 0.9934
